### Installation des packages

In [1]:
%%bash
pip install -q umap-learn
pip install -q keybert
pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.5 MB/s eta 0:00:00


In [4]:
!nvidia-smi

Fri Nov  7 10:08:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from collections import Counter
from typing import List, Dict, Union
from pprint import pprint
import pickle
import requests

from scipy.io import loadmat
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from keybert import KeyBERT
import torch
import umap
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import altair as alt
from sklearn.metrics import confusion_matrix, accuracy_score
from scipy.optimize import linear_sum_assignment

# **Exercice 1**

Utiliser KeyBERT pour extraire les mots-clés de chaque document, puis essayer de représenter les topics par les termes les plus fréquents en ne comptant que les mots qui apparaissent souvent dans les documents. **Comparez ce résultat avec celui obtenu lors du premier TP.**

In [3]:
# Télécharger le fichier directement depuis le lien AWS.
url = "https://cifre.s3.eu-north-1.amazonaws.com/classic4.csv"
data = pd.read_csv(url)
data.head(5)

,text,label,label_id
0,Williams & Wilkins - The Great Leap Backward\n...,cisi,1
1,the transonic characteristics of 38 cambered r...,cran,2
2,7828. selective cerebral hypothermia physiolo...,med,3
3,A Study of Six University-Based Information Sy...,cisi,1
4,Generation of Permutations in Lexicographic Or...,cacm,0


#### Je travaille actuellement avec **un modèle contextuel**, donc je peux me permettre de ne pas effectuer de prétraitement visant à éliminer la ponctuation, les stopwords, etc. De plus, le modèle KeyBERT réalise un certain nettoyage lorsqu'il calcule la matrice de comptage (ou de fréquence) avec le CountVectorizer (notamment avec `stop_words='english'`).


#### J'ai consulté la documentation (https://maartengr.github.io/KeyBERT/guides/quickstart.html#guided-keybert) et je me suis rendu compte qu'il est possible d'accélérer les calculs en précalculant les embeddings avant la génération des mots-clés.

In [5]:
kw_model = KeyBERT(model='all-MiniLM-L6-v2')
texts = data['text'].tolist()
doc_embeddings, word_embeddings = kw_model.extract_embeddings(
    texts,
    stop_words='english',
    min_df=50)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
keywords = kw_model.extract_keywords(texts,
                                     stop_words='english',
                                     min_df=50,
                                     doc_embeddings=doc_embeddings,
                                     word_embeddings=word_embeddings,
                                     top_n=10)
keywords_only = [[w[0] for w in kw] for kw in keywords]
data['keywords'] = keywords_only

In [7]:
# Check la sortie.
data.iloc[1]['keywords']

['aerodynamic',
 'wings',
 'thickness',
 'transonic',
 'characteristics',
 'drag',
 'lift',
 'reynolds',
 'ratios',
 'rectangular']

In [8]:
def get_top_terms_per_topic(df: pd.DataFrame,
                            keywords_column: str,
                            topic_column: str,
                            top_n: int = 20) -> Dict[str, List[str]]:
    """
    Cette fonction retourne les termes les plus fréquents pour chaque topic en
    fonction des mots-clés déjà extraits dans les documents.

    Args:
        df: DataFrame contenant les documents, les identifiants des topics, et
        les mots-clés.
        keywords_column: Nom de la colonne contenant les mots-clés de chaque
        document (sous forme de liste).
        topic_column: Nom de la colonne contenant les identifiants des topics.
        top_n: Nombre de termes les plus importants à extraire par topic.

    Returns:
        Dictionnaire où les clés sont les identifiants des topics et les valeurs
        sont les listes des termes les plus fréquents.
    """

    # Obtenir les topics uniques dans le DataFrame.
    topics = df[topic_column].unique()
    top_terms = {}

    # Boucler sur les topics pour récupérer les top termes.
    for topic in topics:

        # Filtrer les documents appartenant au topic actuel.
        topic_keywords = df[df[topic_column] == topic][keywords_column].tolist()

        # Fusionner les listes de mots-clés des documents dans une seule liste.
        all_keywords = [keyword for sublist in topic_keywords for keyword in
                        sublist]

        # Calculer la fréquence des termes dans les mots-clés.
        keyword_freq = Counter(all_keywords)

        # Extraire les `top_n` termes les plus fréquents.
        most_common_terms = [term for term, freq in keyword_freq.most_common(
            top_n)]
        top_terms[topic] = most_common_terms

    return top_terms


top_terms = get_top_terms_per_topic(data,
                                    keywords_column='keywords',
                                    topic_column='label_id',
                                    top_n=20)
pprint(top_terms)

{np.int64(0): ['algorithm',
               'computer',
               'program',
               'programming',
               'systems',
               'programs',
               'language',
               'data',
               'method',
               'processing',
               'algorithms',
               'storage',
               'implementation',
               'matrix',
               'memory',
               'fortran',
               'function',
               'languages',
               'algol',
               'structure'],
 np.int64(1): ['library',
               'information',
               'research',
               'libraries',
               'retrieval',
               'scientific',
               'systems',
               'data',
               'science',
               'literature',
               'book',
               'indexing',
               'study',
               'search',
               'analysis',
               'documents',
               'document',
       

In [35]:
# Aplatir les mots-clés et conserver l'index du topic (utile pour exo 2).
flattened_keywords = []
topics = []
for topic, words in top_terms.items():
    flattened_keywords.extend(words)
    topics.extend([topic] * len(words))

## **Interprétation**
Les deux méthodes (word2vec et keyBert) semblent avoir identifié des thèmes principaux très similaires, mais le modèle keyBert offre une meilleure cohérence thématique et utilise des termes plus spécifiques qui aident à définir clairement chaque sujet.

# **Exercice 2**

1. Récupérez les vecteurs de ces mots-clés en utilisant le modèle BERT.
2. Appliquez ACP et UMAP sur les vecteurs obtenus et colorez les points en fonction de la colonne `label_id`.
3. Interprétez les résultats et comparez-les avec ceux obtenus avec Word2Vec.

#### Il y a sûrement **des méthodes plus intelligentes pour créer les vecteurs** (comme **passer tout le document au modèle afin qu'il puisse représenter les mots-clés en fonction de leur contexte, puis extraire les vecteurs des tokens qui correspondent aux mots-clés**), mais honnêtement, je n'ai pas envie de me prendre la tête. Alors voilà une solution qui fait le taf, et ça ira très bien !

In [36]:
bert_model = BertModel.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


def get_bert_embeddings_in_batches(keywords: List[str],
                                   batch_size: int = 32) -> np.ndarray:
    """
    Cette fonction prend une liste de mots-clés et renvoie leurs embeddings
    BERT sous forme de vecteurs. Les mots-clés sont traités par batchs pour
    améliorer l'efficacité.

    Args:
        keywords: Liste de mots-clés à encoder.
        batch_size: Taille du batch, définissant combien de mots-clés seront
        traités en parallèle. Default est 32.

    Returns:
          Array d'embeddings pour chaque mot-clé, où chaque embedding est un
          vecteur BERT de taille (768,).
    """

    # Liste pour stocker les embeddings de chaque batch
    embeddings = []

    # Diviser les mots-clés en batchs pour traitement par lots.
    for i in range(0, len(keywords), batch_size):

        # Sélectionner un sous-ensemble de mots-clés (batch).
        batch = keywords[i:i + batch_size]

        # Tokenizer le batch de mots-clés.
        inputs = tokenizer(batch,
                           return_tensors='pt',
                           padding=True,
                           truncation=True,
                           max_length=10)

        # Obtenir les embeddings BERT sans calculer les gradients (en mode inférence).
        with torch.no_grad():
            outputs = bert_model(**inputs)

        # Moyenne des tokens pour les mots qui seront représentés par plusieurs
        # tokens.
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)

        # Ajouter les embeddings du batch à la liste principale.
        embeddings.extend(batch_embeddings)

    # Convertir les embeddings en numpy array.
    return np.array([emb.numpy() for emb in embeddings])


batch_size = 20
keyword_embeddings = get_bert_embeddings_in_batches(flattened_keywords,
                                                    batch_size=batch_size)

# Check les 3 premiers résultats.
for keyword, embedding in zip(flattened_keywords[:3], keyword_embeddings[:3]):
    print(f"Keyword: `{keyword}` ---> Dimension du vecteur: {embedding.shape}")

Keyword: `library` ---> Dimension du vecteur: (768,)
Keyword: `information` ---> Dimension du vecteur: (768,)
Keyword: `research` ---> Dimension du vecteur: (768,)


In [37]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(keyword_embeddings)

umap_model = umap.UMAP(n_components=2)
umap_result = umap_model.fit_transform(keyword_embeddings)

df_pca = pd.DataFrame(pca_result, columns=['x', 'y'])
df_pca['keyword'] = flattened_keywords
df_pca['topic'] = topics

df_umap = pd.DataFrame(umap_result, columns=['x', 'y'])
df_umap['keyword'] = flattened_keywords
df_umap['topic'] = topics

pca_chart = alt.Chart(df_pca).mark_circle(size=100).encode(
    x='x:Q',
    y='y:Q',
    color='topic:N',
    tooltip=['keyword', 'topic']
).properties(
    width=300,
    height=300,
    title="Projection PCA des mots-clés"
).interactive()

umap_chart = alt.Chart(df_umap).mark_circle(size=100).encode(
    x='x:Q',
    y='y:Q',
    color='topic:N',
    tooltip=['keyword', 'topic']
).properties(
    width=300,
    height=300,
    title="Projection UMAP des mots-clés"
).interactive()

In [38]:
# NB : pour afficher les mots-clés, il suffit de survoler les points.
# Le graphique est interactif, vous pouvez zoomer, dézoomer, vous déplacer, etc.
pca_chart | umap_chart

alt.HConcatChart(...)

## **Interprétation de résultats**
La visualisation faite avec l'UMAP confirme que le clustering est de haute qualité. L'UMAP a bien réussi là où la PCA a échoué (on remarque une séparation difficile entre les groupes Bleu et Vert avec la PCA) indique que les distinctions entre les thèmes ne sont pas de nature simple et linéaire, mais sont basées sur des relations non-linéaires plus complexes entre les mots : certains termes génériques sont partagés entre les deux domaines.


## **Comparaison avec les résultats de word2vec**
Avec la PCA comme avec l'UMAP, les résultats significativement mieux : L'extraction des termes, basée sur les embeddings de keyBERT, réussit a séparer les documents en 4 groupes avec frontières bien claires.
Dans le cas de word2vec, les 4 classes s'emboitaient. Ce problème n'a plus lieu avec keyBERT.

# **Exercice 3**

1. Créez les vecteurs des documents en utilisant la somme et la moyenne des tokens.
2. Lancez KMeans sur les deux représentations avec 4 clusters. Veillez bien à augmenter le nombre d'itérations et d'initialisations.
3. Visualisez les clusters formés avec UMAP et ACP, et interprétez les résultats.
4. Visualisez les matrices de confusion et interprétez-les.

In [39]:
# Charger le modèle.
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")


def compute_embeddings(texts: Union[str, List[str]],
                       model: SentenceTransformer) -> Dict[str, np.ndarray]:
    """
    Calcule les embeddings pour un ou plusieurs textes en utilisant Sentence
    Transformers et retourne à la fois la somme et la moyenne des embeddings.

    Args:
        texts: Le texte ou la liste de textes pour lesquels calculer les
          embeddings.
        model: Le modèle Sentence Transformer à utiliser.

    Returns:
        Un dictionnaire contenant les embeddings calculés avec la somme et la
        moyenne pour chaque texte.
            {
               'sum': numpy.ndarray des embeddings par somme,
               'mean': numpy.ndarray des embeddings par moyenne
            }
    """

    # Si un seul texte est passé, le convertir en liste pour uniformiser le
    # traitement.
    if isinstance(texts, str):
        texts = [texts]

    # Extraire les embeddings des tokens de chaque document.
    # Le modèle accepte une liste de textes.
    token_embeddings = model.encode(texts,
                                    output_value='token_embeddings')

    embeddings_sum = []
    embeddings_mean = []

    for tokens in token_embeddings:
        # Move the tensor to CPU before converting to NumPy
        embeddings_sum.append(
            np.sum(tokens.cpu().numpy(), axis=0))  ### Somme sur les tokens.
        embeddings_mean.append(
            np.mean(tokens.cpu().numpy(), axis=0)) ### Moyenne sur les tokens.

    return {
        'sum': np.array(embeddings_sum),
        'mean': np.array(embeddings_mean)
    }

In [40]:
dict_of_embeddings = compute_embeddings(data.text.tolist(), model)

In [55]:
def apply_kmeans_to_embeddings(embeddings: dict,
                               n_clusters: int = 4,
                               max_iter: int = 400,
                               n_init: int = 50,
                               keys: List[str] = ["sum", "mean"]):
    """
    Applique l'algorithme KMeans sur les deux représentations des embeddings
    (somme et moyenne) avec 4 clusters, et retourne les labels obtenus.

    Args:
        embeddings: Un dictionnaire contenant les embeddings calculés.
        n_clusters: Nombre de clusters à utiliser pour KMeans.
        max_iter: Nombre maximal d'itérations pour KMeans.
        n_init: Nombre d'initialisations de l'algorithme pour KMeans.
        keys: Les clés des embeddings à utiliser.

    Returns:
        Un dictionnaire contenant les labels obtenus après clustering pour.
            {
                'sum_labels': Labels pour la représentation par somme,
                'mean_labels': Labels pour la représentation par moyenne
            }
    """

    # Appliquer KMeans sur les embeddings de la somme.
    kmeans_sum = KMeans(n_clusters=n_clusters,
                        max_iter=max_iter,
                        n_init=n_init,
                        random_state=2024)
    sum_labels = kmeans_sum.fit_predict(embeddings[keys[0]])

    # Appliquer KMeans sur les embeddings de la moyenne.
    kmeans_mean = KMeans(n_clusters=n_clusters,
                         max_iter=max_iter,
                         n_init=n_init,
                         random_state=2024)
    mean_labels = kmeans_mean.fit_predict(embeddings[keys[1]])

    return {
        f'{keys[0]}_labels': sum_labels,
        f'{keys[1]}_labels': mean_labels
    }


clusters = apply_kmeans_to_embeddings(dict_of_embeddings,
                                      n_clusters=4,
                                      max_iter=400,
                                      n_init=50)

In [56]:
def reduce_dimensions_for_vis(embeddings: dict, method: str = "pca") -> dict:
    """
    Réduit la dimension des embeddings en utilisant PCA ou UMAP avec 2
    composantes.

    Args:
        embeddings: Dictionnaire contenant les embeddings.
                      - 'sum': contenant les embeddings basés sur la somme.
                      - 'mean': contenant les embeddings basés sur la moyenne.
        method: La méthode de réduction de dimension à utiliser.
                Peut être soit 'pca' (par défaut) soit 'umap'.

    Returns:
        Un dictionnaire contenant les embeddings réduits avec les clés :
          {
              'sum': les embeddings réduits pour la somme,
              'mean': les embeddings réduits pour la moyenne.
          }
    """

    # Choix du réducteur de dimension en fonction de la méthode.
    if method == "pca":
        reducer = PCA(n_components=2)
    elif method == "umap":
        reducer = umap.UMAP(n_components=2, random_state=2024)

    # Appliquer la réduction de dimension.
    reduced_embeddings = {
        'sum': reducer.fit_transform(embeddings['sum']),
        'mean': reducer.fit_transform(embeddings['mean'])
    }

    return reduced_embeddings

In [27]:
!pip install -q  "vegafusion[embed]>=1.6.0" vegafusion>=1.6.0
!pip install -q "vl-convert-python>=1.6.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 34.1 MB/s eta 0:00:00


In [57]:
import altair as alt
alt.data_transformers.enable("vegafusion")

pca_reduced = reduce_dimensions_for_vis(dict_of_embeddings, method="pca")
umap_reduced = reduce_dimensions_for_vis(dict_of_embeddings, method="umap")

df_pca_sum = pd.DataFrame(pca_reduced['sum'], columns=['x', 'y'])
df_pca_sum['sum_labels'] = clusters['sum_labels']
df_pca_sum['type'] = 'sum'

df_pca_mean = pd.DataFrame(pca_reduced['mean'], columns=['x', 'y'])
df_pca_mean['mean_labels'] = clusters['mean_labels']
df_pca_mean['type'] = 'mean'

df_umap_sum = pd.DataFrame(umap_reduced['sum'], columns=['x', 'y'])
df_umap_sum['sum_labels'] = clusters['sum_labels']
df_umap_sum['type'] = 'sum'

df_umap_mean = pd.DataFrame(umap_reduced['mean'], columns=['x', 'y'])
df_umap_mean['mean_labels'] = clusters['mean_labels']
df_umap_mean['type'] = 'mean'


pca_chart = alt.Chart(df_pca_mean).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='mean_labels:N',
    shape='type:N',
    tooltip=['mean_labels', 'type']
).properties(
    width=400,
    height=400,
    title="PCA avec la moyenne"
).interactive()

umap_chart = alt.Chart(df_pca_sum).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='sum_labels:N',
    shape='type:N',
    tooltip=['sum_labels', 'type']
).properties(
    width=400,
    height=400,
    title="PCA avec la somme"
).interactive()

pca_chart | umap_chart

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


alt.HConcatChart(...)

In [44]:
pca_chart = alt.Chart(df_umap_sum).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='sum_labels:N',
    shape='type:N',
    tooltip=['sum_labels', 'type']
).properties(
    width=400,
    height=400,
    title="UMAP avec la somme"
).interactive()

umap_chart = alt.Chart(df_umap_mean).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='mean_labels:N',
    shape='type:N',
    tooltip=['mean_labels', 'type']
).properties(
    width=400,
    height=400,
    title="UMAP avec la moyenne"
).interactive()

pca_chart | umap_chart

alt.HConcatChart(...)

## **Interprétation**
Les résultats indiquent que la représentation par moyenne produit des clusters plus denses, bien séparés et cohérents sur le plan sémantique, car elle atténue l’effet de la longueur des textes. À l’inverse, la représentation par somme engendre des regroupements plus dispersés et moins homogènes, les vecteurs étant amplifiés pour les documents longs. La moyenne des embeddings apparaît donc comme une approche plus stable et pertinente pour la représentation et la classification thématique des documents.

In [45]:
def synchronize_labels(real_labels: List[int],
                       predicted_labels: List[int]) -> List[int]:
    """Applique l'algorithme hongrois pour réordonner les labels basés sur la
    matrice de confusion.

    Cette fonction crée une matrice de confusion entre les labels réels et
    prévus, puis utilise l'algorithme hongrois pour trouver les correspondances
    optimales entre les labels réels et prévus. L'objectif est de maximiser la
    somme des valeurs dans la matrice de confusion pour les paires assignées.

    Args:
        real_labels: Labels de vérité terrain.
        predicted_labels: Une liste de labels prédits.

    Returns:
        Une liste de labels prédits réordonnée basée sur les correspondances
        optimales.
    """

    # Créer la matrice de confusion.
    cm = confusion_matrix(real_labels, predicted_labels)

    # Appliquer l'algorithme hongrois pour maximiser les correspondances.
    row_indices, col_indices = linear_sum_assignment(-cm)

    # Créer une correspondance entre les labels réels et les labels prédits.
    label_mapping = {col: row for row, col in zip(row_indices, col_indices)}

    # Réassigner les labels prédits en utilisant la correspondance trouvée.
    synchronized_labels = [label_mapping[label] for label in predicted_labels]

    return synchronized_labels


def plot_confusion_matrix(true_labels,
                          synchronized_labels,
                          title,
                          topic_names):
    # Create confusion matrix.
    cm = confusion_matrix(true_labels, synchronized_labels)
    cm_df = pd.DataFrame(
        cm, columns=[f'Pred_{i}' for i in range(cm.shape[1])],
        index=[f'True_{i} ({topic_names[i]})' for i in range(cm.shape[0])])

    cm_df_melt = cm_df.reset_index().melt(id_vars='index',
                                          var_name='Predicted',
                                          value_name='Count')
    cm_df_melt.rename(columns={'index': 'True'}, inplace=True)

    accuracy = accuracy_score(true_labels, synchronized_labels)

    heatmap = alt.Chart(cm_df_melt).mark_rect().encode(
        x=alt.X('Predicted:N', title='Predicted Labels'),
        y=alt.Y('True:N', title='True Labels'),
        color=alt.Color('Count:Q', scale=alt.Scale(scheme='blues')),
        tooltip=['True:N', 'Predicted:N', 'Count:Q']
    ).properties(
        title=f'{title} (Accuracy: {accuracy:.4f})',
        width=400,
        height=400
    )

    # Add text labels on the heatmap
    text = heatmap.mark_text(baseline='middle').encode(
        text='Count:Q',
        color=alt.condition(
            alt.datum.Count > 0.5 * cm_df_melt['Count'].max(),
            alt.value('white'),
            alt.value('black')
        )
    )

    return heatmap + text


# Synchroniser les labels.
synchronized_sum_labels = synchronize_labels(data.label_id.tolist(),
                                             clusters['sum_labels'].tolist())
synchronized_mean_labels = synchronize_labels(data.label_id.tolist(),
                                              clusters['mean_labels'].tolist())

plot_sum_labels_altair = plot_confusion_matrix(data.label_id.tolist(),
                                               synchronized_sum_labels,
                                               'Somme des embeds des tokens',
                                               data.label.unique())
plot_mean_labels_altair = plot_confusion_matrix(data.label_id.tolist(),
                                                synchronized_mean_labels,
                                                'Moyenne embeds des tokens',
                                                data.label.unique())

In [46]:
plot_sum_labels_altair | plot_mean_labels_altair

alt.HConcatChart(...)

* **Observation :**
Après la réduction de dimensionalité des données à 50 dimensions via la PCA, le clustering K-Means produit une matrice de confusion presque diagonale (après réaffectation des clusters). Les quatre groupes identifiés correspondent exactement aux thèmes réels (cacm, cisi, cran, med), avec une quasi-absence d’erreurs élevé ( en terme de classification ratée). La réduction dimensionnelle a donc préservé l’information essentielle, permettant un clustering aussi performant que celui obtenu sur les données originales (768D).

# **Exercice 4**
1. Utiliser PCA et UMAP pour réduire la dimension des embeddings avant le clustering.
2. Reprenez les étapes 2, 3 et 4 de l'exercice 3.

#### Étant donné que la somme et la moyenne donnent des résultats similaires, je préfère continuer avec la moyenne pour cet exercice afin de vous faciliter la compréhension (En vérité, j'avais la flemme de faire les deux, alors il me fallait une excuse pour me justifier).

In [47]:
def reduce_dimensions(embeddings: dict,
                      embedding_type: str = "mean") -> dict:
    """
    Réduit la dimension des embeddings en utilisant PCA et UMAP, uniquement sur
    la somme ou la moyenne, selon le type d'embeddings sélectionné.

    Args:
        embeddings: Dictionnaire contenant les embeddings.
                      - 'sum': contenant les embeddings basés sur la somme.
                      - 'mean': contenant les embeddings basés sur la moyenne.
        method: La méthode de réduction de dimension à utiliser.
        embedding_type: Le type d'embeddings à utiliser pour la réduction.
                        Peut être soit 'sum' soit 'mean'.

    Returns:
        Un dictionnaire contenant les embeddings réduits.
    """

    pca_reducer = PCA(n_components=0.95)
    umap_reducer = umap.UMAP(n_components=100,
                             random_state=2024)

    # Appliquer la réduction de dimension uniquement sur le type d'embeddings sélectionné.
    reduced_embeddings = {
        "umap": umap_reducer.fit_transform(embeddings[embedding_type]),
        "pca": pca_reducer.fit_transform(embeddings[embedding_type])
    }

    return reduced_embeddings

In [48]:
reduced_embeddings = reduce_dimensions(embeddings=dict_of_embeddings,
                                       embedding_type="mean")

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [49]:
clusters_after_reduction = apply_kmeans_to_embeddings(
    reduced_embeddings,
    n_clusters=4,
    max_iter=300,
    n_init=50,
    keys = list(reduced_embeddings.keys()))

In [50]:
clusters_after_reduction

{'umap_labels': array([1, 0, 2, ..., 0, 2, 3], dtype=int32),
 'pca_labels': array([0, 3, 1, ..., 3, 1, 3], dtype=int32)}

In [51]:
df_pca_mean["pca_labels"] = clusters_after_reduction["pca_labels"]
df_pca_mean["umap_labels"] = clusters_after_reduction["umap_labels"]
df_umap_mean["pca_labels"] = clusters_after_reduction["pca_labels"]
df_umap_mean["umap_labels"] = clusters_after_reduction["umap_labels"]

In [52]:
pca_chart = alt.Chart(df_pca_mean).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='pca_labels:N',
    shape='type:N',
    tooltip=['pca_labels', 'type']
).properties(
    width=400,
    height=400,
    title="PCA (PCA pré-clustering)"
).interactive()

umap_chart = alt.Chart(df_pca_mean).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='umap_labels:N',
    shape='type:N',
    tooltip=['umap_labels', 'type']
).properties(
    width=400,
    height=400,
    title="PCA (UMAP pré-clustering)"
).interactive()

pca_chart | umap_chart

alt.HConcatChart(...)

## **Interprétation**
Les deux graphiques comparent l’effet de la réduction de dimension par ACP et UMAP avant le clustering. Avec l’ACP (graphique de gauche), les clusters apparaissent séparés mais aux frontières floues, présentant plusieurs chevauchements. En revanche, l’UMAP (graphique de droite) produit des groupes plus compacts et nettement mieux délimités. Cela montre que l’approche non linéaire d’UMAP capture plus efficacement la structure intrinsèque des embeddings, offrant une visualisation plus claire et des regroupements plus cohérents que la méthode linéaire de l’ACP.

In [53]:
pca_chart = alt.Chart(df_umap_mean).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='pca_labels:N',
    shape='type:N',
    tooltip=['pca_labels', 'type']
).properties(
    width=400,
    height=400,
    title="UMAP (PCA pré-clustering)"
).interactive()

umap_chart = alt.Chart(df_umap_mean).mark_circle(size=50).encode(
    x='x:Q',
    y='y:Q',
    color='umap_labels:N',
    shape='type:N',
    tooltip=['umap_labels', 'type']
).properties(
    width=400,
    height=400,
    title="UMAP (UMAP pré-clustering)"
).interactive()

pca_chart | umap_chart

alt.HConcatChart(...)

## *Analyse**
Les deux matrices de confusion mettent en évidence la supériorité de l’approche non linéaire UMAP par rapport à l’ACP pour le prétraitement des embeddings avant le clustering K-Means.
L’exactitude du modèle est nettement plus élevée avec UMAP (0,8152 soit 81,52 %) qu’avec l’ACP (0,7352 soit 73,52 %). L’amélioration la plus marquée concerne le cluster True_0 (cisl), où l’ACP confond 1679 échantillons avec Pred_1, une erreur largement corrigée par UMAP. Ces résultats montrent que la structure des données est non linéaire et mieux préservée par UMAP, permettant à K-Means de générer des clusters plus cohérents et plus proches des étiquettes réelles.

In [54]:
# Synchroniser les labels.
synchronized_pca_labels = synchronize_labels(
    data.label_id.tolist(),
    clusters_after_reduction['pca_labels'].tolist())
synchronized_umap_labels = synchronize_labels(
    data.label_id.tolist(),
    clusters_after_reduction['umap_labels'].tolist())

plot_pca_labels_altair = plot_confusion_matrix(data.label_id.tolist(),
                                               synchronized_pca_labels,
                                               'PCA avant KMeans',
                                               data.label.unique())
plot_umap_labels_altair = plot_confusion_matrix(data.label_id.tolist(),
                                                synchronized_umap_labels,
                                                'UMAP avant KMeans',
                                                data.label.unique())

plot_pca_labels_altair | plot_umap_labels_altair

alt.HConcatChart(...)